In [75]:
import pandas as pd
import matplotlib as mpl
import random
import json
import folium
import numpy as np
import matplotlib.pyplot as plt

In [142]:
# 최종 dataframe 후보 1을 가져오자. 
dataframe_raw = pd.read_csv("attribute_선별한 dataframe_초안_1.csv", encoding='utf-8')
dataframe = dataframe_raw.copy()

In [143]:
# 필요없는 column을 drop하자.
dataframe.drop(columns='Unnamed: 0', inplace=True)

In [144]:
dataframe

,자치구,동,edu_percent,동별_친환경자동차수,합계_사업체수,㎡당 매매평균가,행정구역코드,가구수별_승용차수,동별_수소차_대수,택시
0,종로구,사직동,8319.397863,95.695177,3544,756.00,1101053.0,0.144169,0.569495,1.653815e+09
1,종로구,삼청동,2603.942721,29.952259,753,716.00,1101054.0,0.150556,0.170688,1.689370e+09
2,종로구,부암동,8928.650976,102.703207,567,345.00,1101055.0,0.147209,0.598577,1.597098e+09
3,종로구,평창동,16188.846626,186.214745,739,421.00,1101056.0,0.164524,0.971080,1.496236e+09
4,종로구,무악동,6999.208224,80.509489,531,809.00,1101057.0,0.164636,0.419559,1.654529e+09
5,종로구,교남동,9190.485763,105.715003,347,1065.75,1101058.0,0.318773,0.284528,1.771692e+09
6,종로구,가회동,3852.106609,44.309460,747,716.00,1101060.0,0.132454,0.287013,1.623939e+09
7,종로구,종로1.2.3.4가동,7170.375302,82.478365,15851,511.00,1101061.0,0.117474,0.602376,1.642308e+09
8,종로구,종로5.6가동,4742.683830,54.553464,7613,511.00,1101063.0,0.119375,0.392085,1.661424e+09
9,종로구,이화동,7305.105824,84.028124,1289,729.00,1101064.0,0.112555,0.640518,1.655889e+09


In [145]:
dataframe.drop(columns='택시', inplace=True)

# 각 DATAFRAME의 attribute를  interval로 나누자

#### 1. 각 attribute를 분포를 고려하여 5등분한다. (높은 것을 5로 해서 5 ~ 1까지로 한다)
#### 2. 따로 dataframe을 만들어서 위에서 구한 attribute를 저장한다. 
#### 3. 각 점수를 모두 더한 총 점수를 만들고 5등분 한다. 
#### 4. 총 점수를 5등분한 결과값을 가지고 folium으로 visualize한다. 

In [146]:
# interval를 저장할 dataframe을 만든다. 
dataframe_interval_raw = pd.DataFrame(columns=dataframe.columns)
dataframe_interval = dataframe_interval_raw.copy()

### 1. 각 attribute의 분포를 고려하여 5등분한다. 
### 2. 따로 dataframe을 만들어서 위에서 구한 attribute를 저장한다. 

In [149]:
drop_column = [] # 만약 error가 나는 애들이 있다면 column이름을 저장하는 리스트이다. 

for column in dataframe_interval.columns:
    
    if type(dataframe[column].iloc[0]) is str: # 만약 column의 값이 문자열이라면, ex) 자치구, 동
        pass
    else:
        try:
            dataframe_interval[column]=pd.qcut(dataframe[column],60, labels = list(range(1,61))) # 값을 5등분으로 나누고 값을 1,2,3,4,5로 준다. 
        except Exception as e:#만약 error나 나면 처리한다. 
            print(e)
            print(column)
            drop_column.append(column) # error가 나는 column의 이름을 저장한다. 
            dataframe_interval[column]=pd.qcut(dataframe[column],5, duplicates='drop')    
            pass

In [150]:
# 백화점_개소와 store_총_개소는 value가 너무 없어서, 삭제 한다. 
dataframe_interval.drop(columns=drop_column, inplace=True) 

In [151]:
# 자치구와 동에 값을 넣다. 
# 나중에 잘린 값들이 실제 값들과 일치하는 지 체크해야 한다. 
dataframe_interval['자치구'] = dataframe['자치구']
dataframe_interval['동'] = dataframe['동']

In [152]:
dataframe_interval

,자치구,동,edu_percent,동별_친환경자동차수,합계_사업체수,행정구역코드,가구수별_승용차수,동별_수소차_대수
0,종로구,사직동,5,6,54,1,42,53
1,종로구,삼청동,1,1,9,1,43,25
2,종로구,부암동,5,8,4,1,42,53
3,종로구,평창동,21,33,8,1,46,56
4,종로구,무악동,3,4,4,1,46,48
5,종로구,교남동,6,9,1,1,56,40
6,종로구,가회동,1,1,9,1,40,40
7,종로구,종로1.2.3.4가동,4,4,60,1,37,53
8,종로구,종로5.6가동,2,2,59,2,37,47
9,종로구,이화동,4,5,26,2,35,54


### 3. 각 점수를 모두 더한 총 점수를 만들고 5등분 하여 다시 저장한다. 

In [153]:
# 총_점수 column을 만들고 dummy값을 넣는다. 
dataframe_interval['총_점수'] = range(424)

In [154]:
# count할 수 있는 column의 이름만 구한다. 
count_column = list(dataframe_interval.columns)
count_column.remove('자치구')
count_column.remove("동")
count_column.remove("총_점수")

In [155]:
sum_list = [] # 각 column의 점수를 담는 list이다. 

for i in range(len(dataframe_interval)): 
    score_sum = 0 #score_sum은 각 점수를 더하는 변수.
    
    for column in count_column:
        score_sum += dataframe_interval[column].iloc[i] # 각 column의 점수를 더한다. 
    
    sum_list.append(score_sum)

In [156]:
dataframe_interval['총_점수'] = sum_list # 총_점수 column에 더한 총 점수의 list를 준다. 

In [157]:
dataframe_interval['edu_percent']

0       5
1       1
2       5
3      21
4       3
5       6
6       1
7       4
8       2
9       4
10      2
11      5
12      3
13      2
14      5
15      9
16     24
17      1
18      2
19      1
20      1
21      2
22      2
23      1
24      3
25      6
26      6
27      3
28      8
29     12
       ..
394     1
395    55
396    34
397    52
398    56
399    50
400    51
401    52
402    22
403     7
404    57
405    56
406    50
407     6
408    28
409    19
410    42
411     3
412    11
413    21
414    45
415    44
416    25
417    38
418    34
419     1
420    43
421    55
422    54
423    59
Name: edu_percent, Length: 424, dtype: category
Categories (60, int64): [1 < 2 < 3 < 4 ... 57 < 58 < 59 < 60]

In [158]:
dataframe_interval['총_점수_interval'] = pd.qcut(dataframe_interval['총_점수'], 5, labels = [1,2,3,4,5]) #총_점수를 또 5등분한다. 

In [159]:
dataframe_interval

,자치구,동,edu_percent,동별_친환경자동차수,합계_사업체수,행정구역코드,가구수별_승용차수,동별_수소차_대수,총_점수,총_점수_interval
0,종로구,사직동,5,6,54,1,42,53,161,3
1,종로구,삼청동,1,1,9,1,43,25,80,1
2,종로구,부암동,5,8,4,1,42,53,113,1
3,종로구,평창동,21,33,8,1,46,56,165,3
4,종로구,무악동,3,4,4,1,46,48,106,1
5,종로구,교남동,6,9,1,1,56,40,113,1
6,종로구,가회동,1,1,9,1,40,40,92,1
7,종로구,종로1.2.3.4가동,4,4,60,1,37,53,159,3
8,종로구,종로5.6가동,2,2,59,2,37,47,149,2
9,종로구,이화동,4,5,26,2,35,54,126,2


### 4. 총 점수를 5등분한 결과값을 가지고 folium으로 visualize한다.

In [160]:
# 필요한 geo_data 가져오기.
geo_path = "geoData_copy.json"
geo_str = json.load(open(geo_path, encoding="utf-8"))

In [161]:
# seoul_map_car라는 map을 만들고 Chropleth 만들기.
nested_seoul_map_scored = folium.Map(location=[37.5502, 126.982], zoom_start=10.5, tiles="cartodbpositron")

folium.Choropleth(geo_data=geo_str, 
                     data = dataframe_interval,
                     columns=['동', '총_점수_interval'],
                     fill_color='PuRd', 
                     key_on='properties.name',
                     highlight=True,
                     fill_opacity=0.5,
                     line_opacity=1,
                     legend_name="총_점수_interval").add_to(nested_seoul_map_scored)

# nested_seoul_map_scored.save("Visualization/nested_seoul_map_scored_without_taxi.html")

# DATAFRAME의 총 점수_Interval을 10등분 해보자.

#### 1. 현재 5등분으로 나누었을 때, 생각보다 data들이 뭉쳐있는 것을 볼 수 있었다. 그래서, 5등분하고 10등분으로 나누자. 
#### 2. 그리고 시각화해서 보자. 
#### 3. 충전소의 위치를 찍어보자.

In [162]:
# 10등분으로 나누자.
dataframe_interval['총_점수_interval_10'] = pd.qcut(dataframe_interval['총_점수'], 60, labels = list(range(1,61))) #총_점수를 또 5등분한다. 

In [163]:
dataframe_interval

,자치구,동,edu_percent,동별_친환경자동차수,합계_사업체수,행정구역코드,가구수별_승용차수,동별_수소차_대수,총_점수,총_점수_interval,총_점수_interval_10
0,종로구,사직동,5,6,54,1,42,53,161,3,27
1,종로구,삼청동,1,1,9,1,43,25,80,1,3
2,종로구,부암동,5,8,4,1,42,53,113,1,11
3,종로구,평창동,21,33,8,1,46,56,165,3,27
4,종로구,무악동,3,4,4,1,46,48,106,1,9
5,종로구,교남동,6,9,1,1,56,40,113,1,11
6,종로구,가회동,1,1,9,1,40,40,92,1,5
7,종로구,종로1.2.3.4가동,4,4,60,1,37,53,159,3,26
8,종로구,종로5.6가동,2,2,59,2,37,47,149,2,23
9,종로구,이화동,4,5,26,2,35,54,126,2,15


### 2. 시각화해서 보자.

In [164]:
# seoul_map_car라는 map을 만들고 Chropleth 만들기.
nested_seoul_map_scored_10 = folium.Map(location=[37.5502, 126.982], zoom_start=10.5, tiles="cartodbpositron")

folium.Choropleth(geo_data=geo_str, 
                     data = dataframe_interval,
                     columns=['동', '총_점수_interval_10'],
                     fill_color='PuRd', 
                     key_on='properties.name',
                     highlight=True,
                     fill_opacity=0.5,
                     line_opacity=1,
                     legend_name="총_점수_interval_10").add_to(nested_seoul_map_scored_10)

# nested_seoul_map_scored_10.save("Visualization/nested_seoul_map_scored_10.html")

In [165]:
# dataframe_interval을 csv로 다운로드하자. 
# dataframe_interval.to_csv("modified_Data/중첩분석을 위한_final_dataframe.csv", encoding='utf-8')

In [166]:
dataframe.columns

Index(['자치구', '동', 'edu_percent', '동별_친환경자동차수', '합계_사업체수', '㎡당 매매평균가',
       '행정구역코드', '가구수별_승용차수', '동별_수소차_대수'],
      dtype='object')

In [167]:
# 필요없는 column지우고 csv로 저장하자. 
dataframe.drop(columns=['store_총_개소', '백화점_개소'], inplace=True)

KeyError: "['store_총_개소' '백화점_개소'] not found in axis"

In [137]:
dataframe.to_csv("modified_Data/attribute_선별한 dataframe_초안_1.csv", encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: 'modified_Data/attribute_선별한 dataframe_초안_1.csv'

## 3. 충전소의 위치를 찍어보자

In [168]:
dataframe_interval = pd.read_csv("중첩분석을 위한_final_dataframe.csv", encoding='utf-8')

In [169]:
# seoul_map_car라는 map을 만들고 Chropleth 만들기.
nested_seoul_map_scored_10 = folium.Map(location=[37.5502, 126.982], zoom_start=10.5, tiles="cartodbpositron")

folium.Choropleth(geo_data=geo_str, 
                     data = dataframe_interval,
                     columns=['동', '총_점수_interval_10'],
                     fill_color='PuRd', 
                     key_on='properties.name',
                     highlight=True,
                     fill_opacity=0.5,
                     line_opacity=1,
                     legend_name="총_점수_interval_10").add_to(nested_seoul_map_scored_10)

In [170]:
# 양재 수소충전소를 찍어보자.
folium.Marker(
  location=[37.4686275,127.0319203],
  popup='양재 수소 충전소 위치.',
  icon=folium.Icon(color='red',icon='star')
).add_to(nested_seoul_map_scored_10)

# 상암 수소 충전소를 찍어보자.
folium.Marker(
  location=[37.5687204,126.8761933],
  popup='상암 수소 충전소 위치.',
  icon=folium.Icon(color='red',icon='star')
).add_to(nested_seoul_map_scored_10)

# 국회 수소 충전소를 찍어보자.
folium.Marker(
  location=[37.5311111,126.9132037],
  popup='2019년 완공 예정 국회 수소 충전소 위치.',
  icon=folium.Icon(color='blue',icon='star')
).add_to(nested_seoul_map_scored_10)

# 강동구 상일충전소를 찍어보자.
folium.Marker(
  location=[37.5392486,127.128611],
  popup='2019년 완공 예정 강동구 상일충전소 위치',
  icon=folium.Icon(color='blue',icon='star')
).add_to(nested_seoul_map_scored_10)

# 강서공영차고지 수소생산기지 찍어보자.
folium.Marker(
  location=[37.5775641,126.7963879],
  popup=' 2020년 완공 예정 강서공영차고지 수소생산기지 버스용',
  icon=folium.Icon(color='green',icon='star')
).add_to(nested_seoul_map_scored_10)

# 탄천물재생센터 수소춘전소 찍어보자.
folium.Marker(
  location=[37.4915688,127.0671297],
  popup=' 2020년 완공 예정이지만 미정 탄천물재생센터 수소춘전소',
  icon=folium.Icon(color='green',icon='star')
).add_to(nested_seoul_map_scored_10)

# 현대자동차 계동사옥 수소춘전소 찍어보자.
folium.Marker(
  location=[37.566758,126.9584211],
  popup=' 아직 미정 현대자동차 계동사옥 수소춘전소',
  icon=folium.Icon(color='green',icon='star')
).add_to(nested_seoul_map_scored_10)

In [171]:
nested_seoul_map_scored_10

In [41]:
dataframe_interval.head()

,Unnamed: 0,자치구,동,edu_percent,동별_친환경자동차수,합계_사업체수,㎡당 매매평균가,행정구역코드,가구수별_승용차수,동별_수소차_대수,총_점수,총_점수_interval,총_점수_interval_10
0,0,종로구,사직동,1,1,5,4,1,4,5,21,3,6
1,1,종로구,삼청동,1,1,1,4,1,4,3,15,1,2
2,2,종로구,부암동,1,1,1,1,1,4,5,14,1,2
3,3,종로구,평창동,2,3,1,1,1,4,5,17,2,3
4,4,종로구,무악동,1,1,1,4,1,4,4,16,2,3


# 동별 수소충전소 위치

In [60]:
dataframe_interval[(dataframe_interval['동'].str.contains('천연'))]

,Unnamed: 0,자치구,동,edu_percent,동별_친환경자동차수,합계_사업체수,㎡당 매매평균가,행정구역코드,가구수별_승용차수,동별_수소차_대수,총_점수,총_점수_interval,총_점수_interval_10,수소충전소
192,192,서대문구,천연동,2,2,2,4,3,3,4,20,3,5,0


In [47]:
dataframe_interval['수소충전소']=0

In [61]:
dataframe_interval.loc[dataframe_interval.동.str.contains('양재2동'), '수소충전소'] = 1
dataframe_interval.loc[dataframe_interval.동.str.contains('상암동'), '수소충전소'] = 1
dataframe_interval.loc[dataframe_interval.동.str.contains('여의동'), '수소충전소'] = 1
dataframe_interval.loc[dataframe_interval.동.str.contains('천호3동'), '수소충전소'] = 1
dataframe_interval.loc[dataframe_interval.동.str.contains('방화2동'), '수소충전소'] = 1
dataframe_interval.loc[dataframe_interval.동.str.contains('대치2동'), '수소충전소'] = 1
dataframe_interval.loc[dataframe_interval.동.str.contains('천연동'), '수소충전소'] = 1

In [62]:
dataframe_interval.to_csv('interval.csv')

In [43]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [42]:
rms = sqrt(mean_squared_error(y_actual, y_predicted))

NameError: name 'y_actual' is not defined